In [ ]:
# math for mathematical functions
import math

# numpy for linear algebra and sampling
import numpy as np

# pandas for data processing
import pandas as pd 

# keras for neural network
from keras.models import load_model
from keras import backend

# Tensorflow for neural nerwork model gradients
import tensorflow as tf

# matplotlib for plotting
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap

# sklearn for k-means clustering
from sklearn.cluster import KMeans

# sklearn for k-nearest neighbors
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors, datasets

# sklearn for balltree
from sklearn.neighbors import BallTree

# for k-d tree
from scipy import spatial

# scipy for optimization library
from scipy.optimize import minimize
from scipy.optimize import linprog

# itertools for generating scenarios
import itertools

# time for timing methods
import time

# Import population data
df_florida = pd.read_csv('CleanData/Florida-US-CENSUS-and-Voter-DATA.csv')
# Drop fist column (as it is 'Unnamed' column not containing real data)
df_florida.drop(df_florida.columns[0],axis=1,inplace=True)
# Save header names for data frame
df_florida_header_list = [i for i in list(df_florida.columns.values) if i not in ['County']]

In [ ]:
# -------------------------- Functions associated with Redistricting ------------------------ #

# Function for generating possibly non-contiguous district
def construct_district(n_dist, n_neigh, pts, P, R, p_norm):
    # Create copy of P so P does not get destroyed
    Q = P[:]
    
    # Create copy of R so R does not get destroyed
    S = R[:]
    
    # Initialize n_race
    n_race = len(S)
    
    # Create array of means of population race distributions
    mu = []
    for i in range(n_race):
        mu = np.append(mu, np.mean(S[i]))    
        
    # Initialize mean of race distribution for pts
    R_means = np.zeros((n_dist, n_race))
    
    # Initialize counter for each district
    n_pts = np.ones(n_dist, dtype = int)
    
    # Initialize list of remaining indices
    rem_indices = [i for i in range(len(P))]
    
    # Initialize x_dist, y_dist, and ind_dist used to store x and y coords of pts in each district
    # and the indices of P belonging to each district
    x_dist = [[] for i in range(n_dist)]
    y_dist = [[] for i in range(n_dist)]
    ind_dist = [[] for i in range(n_dist)]
    
    while len(Q) > n_neigh:
        for i in range(n_dist):
            # Create KDTree
            tree = BallTree(Q) #, leaf_size = 100)
            
            # Set number of neighbors to use in update of indices and distances 
            # (need to make sure we don't exceed avaiable number of points left)
            neighbors = min(n_neigh,len(Q)-1) 
            # Determine distances and indices from nearest neighbor search
            dist, indices = tree.query(pts[i], k=neighbors)
            
            # Initialize index of point
            indices = indices.flatten()
            ind = indices[0]
            
            # Initialize min_dist
            min_dist = 100*n_race
            # Initialize temp mean array
            temp_mean = np.zeros(n_race)

            # Determine index which minimizes distances from mean of race distributions
            for j in range(neighbors):
                temp_ind = indices[j]
                # Update race means for test point
                for k in range(n_race):
                    temp_mean[k] = ((n_pts[i]-1)*R_means[i][k] + S[k][temp_ind])/(n_pts[i])
                # Compute distance to mean for each race and sum
                temp_dist = np.mean(temp_mean - mu)
                # Check if distance from mean is smaller than current smallest value
                if temp_dist < min_dist:
                    # Update index
                    ind = temp_ind
                    # Update min_dist
                    min_dist = temp_dist  
                    # Update mean
                    new_means = np.copy(temp_mean)
                    
            # Update R_means
            R_means[i] = np.copy(new_means)
            
            # Append x and y values to district lists
            x_dist[i] = np.append(x_dist[i], Q[ind][0])
            y_dist[i] = np.append(y_dist[i], Q[ind][1])
            
            # Appending indices to index list
            ind_dist[i] = np.append(ind_dist[i], rem_indices[ind])

            # Update counter
            n_pts[i] = n_pts[i] + 1
            
            # Update pts[i]
            #for j in range(len(pts[i])):
            #    pts[i][j] = np.sum(pts[i][j])/n_pts[i]
            
            # Update Q by removing point added to district
            Q = np.delete(Q, ind, 0)
            
            # Update S by removing point added to district
            for k in range(n_race):
                S[k] = np.delete(S[k], ind, 0)
            
            # Update remaining indices by removing used index
            rem_indices = np.delete(rem_indices, ind)
            
    # Return x_dist and y_dist and exit function
    return x_dist, y_dist, ind_dist, R_means

# Function for creating array with number of points per county in each district
def county_per_district(n_dist, n_counties, county_arr, ind_arr):
    # Create array containing number of points from each county in each district
    Y_counties = np.zeros((n_dist, n_counties), dtype = int)
    
    j = len(ind_arr)
    k = len(county_arr)
    print("j,k: %lg, %lg" %(j,k))
    
    # Fill array with appropriate values
    for i in range(len(ind_arr)):
        Y_counties[int(ind_arr[i])][int(county_arr[i])] = Y_counties[int(ind_arr[i])][int(county_arr[i])] + 1
        
    # Return Y_counties
    return Y_counties

# Function for creating district data based on US CENSUS and Voter Registration Data for counties
def create_district_data(X_pops, X_medians, Y_counties, n_dist, n_counties, pop_scale):        
    # Initialize array to store features for districts
    Y_pops = np.zeros((n_dist, np.shape(X_pops)[1]))
    Y_medians = np.zeros((n_dist, np.shape(X_medians)[1]))
    
    # Update medians by averaging medians
    Y_medians = np.matmul(Y_counties, X_medians)
    count_per_dist = np.matmul(Y_counties, np.ones(n_counties))
    
    # Fill population arrays
    for j in range(n_dist):
        Y_medians[j] = Y_medians[j] / (count_per_dist[j])
        #temp = np.zeros(np.shape(X_medians)[1])
        # Update populations and temp 
        for i in range(n_counties):
            # Update populations
            Y_pops[j] = Y_pops[j] + (Y_counties[j][i] / pop_scale) * X_pops[i]
            # Update temp for averaging medians
            #temp = temp + Y_counties[j][i] * X_medians[i]
            
    # Return arrays containing population of each district and medians of each district
    return Y_pops, Y_medians 



# ------------------- Functions for Classifying and Plotting Districts ------------------- #
# ------------ Initial classification of districts ------- #
def initial_district_classification(P, indk, district_number):
    # ------------------- Train nearest neighbors model for classification to enforce contiguity districts --------------
    # Our features will be the set of ordered pairs P
    X_train = P
    # The target data will be the classes determined by partitioning of the points using the construct_districts function
    Y_train = np.zeros(len(P), dtype = int)
    for i in range(num_districts):
        for j in range(len(indk[i])):
            t = int(indk[i][j])
            Y_train[t] = i    

    # --------------- Initial Classification and View of Districts ------------------ #
    # Set the step size in the mesh (large since data points are fairly spread out)
    h = 2500

    # Set number of neighbors to use (larger number seems to force regions to be contiguous)
    n_neighbors = 200

    # Set norm to use when computing distance (using euclidean norm, seems to work well enough)
    nrm = 2

    # Create color maps for the classes
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    for weights in ['distance', 'uniform']:
        # we create an instance of Neighbours Classifier and fit the data.
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights, p=nrm)
        clf.fit(X_train, Y_train)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
        y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        plt.figure(figsize=(15,15))
        plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
        plt.fill([501000,600000,600000,501000], [201000,201000,700000,700000], 'w', edgecolor='none')

        # Plot also the training points
        plt.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, cmap=cmap_bold, edgecolor='k', s=20)
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
        plt.title("3-Class classification (k = %i, weights = '%s')" % (n_neighbors, weights))

        image_name = 'Finalized-Districts/Initial-Images/Districts-' + str(district_number) + '-' + weights + '.png'
        plt.savefig(image_name)

    plt.show()
    
    # Return nearest neighbors model which used uniform distances
    return clf

# ------------ Final classification of districts ------- #
def final_district_classification(P, clf, district_number):
    # ------------------ Use Nearest Neighbors Classifier to Predict district for each population point ---------------- #
    # Our features will be the set of ordered pairs P
    X_train = P   
    # Use the model to finalize districts        
    Y_dist = clf.predict(X_train)
    
    # Set number of neighbors to use (larger number seems to force regions to be contiguous)
    n_neighbors = 200

    # Initialize Districts and Races arrays for storing classified points
    Districts = [[] for i in range(num_districts)]
    Races = [[] for i in range(num_districts)]
    for i in range(num_districts):
        Races[i] = [[] for j in range(num_races)]

    # Set points and race percentages in each district
    for i in range(len(Y_dist)):
        for j in range(num_districts):
            if j == Y_dist[i]:
                Districts[j] = np.append(Districts[j], X_train[i])
                for k in range(num_races):
                    Races[j][k] = np.append(Races[j][k], R[k][i])
                break;

    # Set filename for writing data
    file_name = 'Finalized-Districts/Race-Distributions/Districts-' + str(district_number) + '-race-distribution.txt'
    file = open(file_name, 'w')
                
    # Print statistics for state population
    print("-------- Percentage of each Race across State --------")
    file.write("-------- Percentage of each Race across State --------\n")
    for i in range(num_races):
        print(" Percentage of Race %i:           %lg" %(i, mu_R[i]))
        file.write(" Percentage of Race %i:           %lg\n" %(i, mu_R[i]))
        file.write

    print("\n")

    # Determine race means and population distributions of new districts and print
    for i in range(num_districts):
        print("---------------- District %i ----------------" %(i))
        file.write("---------------- District %i ----------------\n" %(i))
        print(" Population:                     %lg" %(len(Districts[i])))
        file.write(" Population:                     %lg\n" %(len(Districts[i])))
        for j in range(num_races):
            temp = np.mean(Races[i][j])
            print(" Percentage of Race %i:           %lg" %(j, temp))
            file.write(" Percentage of Race %i:           %lg\n" %(j, temp))
            print(" Deviation from Mean for Race %i: %lg" %(j, abs(temp - mu_R[j])))
            file.write(" Deviation from Mean for Race %i: %lg\n" %(j, abs(temp - mu_R[j])))
            
    # Close file
    file.close()

    # -------------------- Final Plot of Districts and Population Denisity Points ---------------------- #
    # Set the step size in the mesh
    h = 2500

    # Create color maps for the classes
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    for i in range(2):
        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
        y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        plt.figure(figsize=(15,15))
        plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
        plt.fill([501000,600000,600000,501000], [201000,201000,700000,700000], 'w', edgecolor='none')

        # Plot also the training points
        if i == 0:
            plt.scatter(X_train[:, 0], X_train[:, 1], c=Y_dist, cmap=cmap_bold, edgecolor='k', s=20)
            plt.xlim(xx.min(), xx.max())
            plt.ylim(yy.min(), yy.max())
            plt.title("3-Class classification (k = %i, weights = 'uniform')" % (n_neighbors))
            image_name = 'Finalized-Districts/Final-Images/District-' + str(district_number) + '-uniform.png'
        else:
            #ax2 = plt.add_subplot(111, aspect='equal')
            #for p in [
            #    patches.Rectangle((0, 0), 20000, 20000, fill=False),
            #    patches.Rectangle((20000, 0), 20000, 20000, fill=False),
            #]:
            #    plt.add_patch(p)
            image_name = 'Finalized-Districts/Final-Images/Districts-' + str(district_number) + '-no-pts.png'
        
        # Save image
        plt.savefig(image_name)

    plt.show()
    
    return Y_dist



# ----------------- Function for generating district data from county data for objective function ---------------------
def county_to_district(df_florida, df_florida_header_list, num_districts, num_counties, county_array, Y_dist, district_number):
    # Generate number of counties per district
    X_counties = county_per_district(num_districts, num_counties, county_array, Y_dist)
    #print(X_counties)

    # Create matrix of values from data frame corresponding to features encoding population values
    X_pops = df_florida.drop(['County', 'Total; Estimate; Median Household income (dollars)', 
                              'Native; Estimate; Median Household income (dollars)', 
                              'Foreign born; Estimate; Median Household income (dollars)', 
                              'Foreign born; Naturalized citizen; Estimate; Median Household income (dollars)', 
                              'Foreign born; Not a U.S. citizen; Estimate; Median Household income (dollars)'], axis=1).values

    # Create matrix of values from data frame corresponding to features encoing median values
    X_medians = df_florida[['Total; Estimate; Median Household income (dollars)', 
                            'Native; Estimate; Median Household income (dollars)', 
                            'Foreign born; Estimate; Median Household income (dollars)', 
                            'Foreign born; Naturalized citizen; Estimate; Median Household income (dollars)', 
                            'Foreign born; Not a U.S. citizen; Estimate; Median Household income (dollars)']].values

    # Generate population and median district data
    District_pops, District_medians = create_district_data(X_pops, X_medians, X_counties, num_districts, num_counties, pop_scale)

    # ---------------- Convert Population values to percentages (configure inputs for neural network) ---------------- #
    # Concatenate population and median matrices (containing district information)
    X_complete = np.concatenate((District_pops, District_medians), axis = 1)

    # Create data frame from district data
    df_florida_p = pd.DataFrame(data=np.int_(X_complete), index=range(X_complete.shape[0]), columns=df_florida_header_list)

    # Update values in data frame to reflect percentages so they can be used in neural network
    # List of headers to exclude as they do not correspond to percentage values
    drop_list = ['Total; Estimate; Total population', 'Native; Estimate; Total population', 
                 'Foreign born; Estimate; Total population', 
                 'Foreign born; Naturalized citizen; Estimate; Total population', 
                 'Foreign born; Not a U.S. citizen; Estimate; Total population',
                 'Total; Estimate; Median Household income (dollars)', 
                 'Native; Estimate; Median Household income (dollars)', 
                 'Foreign born; Estimate; Median Household income (dollars)', 
                 'Foreign born; Naturalized citizen; Estimate; Median Household income (dollars)', 
                 'Foreign born; Not a U.S. citizen; Estimate; Median Household income (dollars)']

    # List of headers of df_fl with values representing percentages
    percentage_list = [i for i in df_florida_header_list if i not in drop_list]

    # Create list of header types used in converting raw population values to percentages
    header_types = ['Total; Estimate;', 'Native; Estimate;', 'Foreign born; Estimate;', 
                    'Foreign born; Naturalized citizen; Estimate;', 'Foreign born; Not a U.S. citizen; Estimate;']

    # Convert each value from raw value to decimal (representing percentage)
    for mystr in percentage_list:
        for header in header_types:
            if header in mystr:
                # Set 'totalstr' to name of header for total population
                totalstr = header + ' Total population'
                # Convert percentage to value
                df_florida_p[mystr] = df_florida_p[mystr] / df_florida_p[totalstr]
                break;

    # ----------------------- Convert raw population values to percentages --------------------- #
    pop_list = ['Active Democrat Voters', 'Active Republican Voters', 'Active Voters with No Party',
                'Native; Estimate; Total population', 
                'Foreign born; Estimate; Total population', 
                'Foreign born; Naturalized citizen; Estimate; Total population', 
                'Foreign born; Not a U.S. citizen; Estimate; Total population']

    # Initialize string for data frame column containing total population values per district
    totalpopstr = 'Total; Estimate; Total population'

    for mystr in pop_list:
        # Convert population to percentage of population in the district
        df_florida_p[mystr] = df_florida_p[mystr] / df_florida_p[totalpopstr]

    # Drop total population percentage as it is not needed anymore
    df_florida_p.drop([totalpopstr], axis = 1, inplace = True)


    # ------------------ Convert median income values to percentages of $100000 ---------------- #
    income_list = ['Total; Estimate; Median Household income (dollars)', 
                   'Native; Estimate; Median Household income (dollars)', 
                   'Foreign born; Estimate; Median Household income (dollars)', 
                   'Foreign born; Naturalized citizen; Estimate; Median Household income (dollars)', 
                   'Foreign born; Not a U.S. citizen; Estimate; Median Household income (dollars)']

    for mystr in income_list:
        # Convert population to percentage of population in the district
        df_florida_p[mystr] = df_florida_p[mystr] / 100000

    # Create matrix of district values to be used in objective function
    X_district = df_florida_p.values

    # Preview values
    df_florida_p.head()
    
    # Remaining values required for objective function
    rem_vars = ['Republican Candidate Spending', 'Democrat Candidate Spending', 
                      'Party in Office', 'Incumbent Republican Candidate Running', 
                      'Incumbent Democrat Candidate Running', 'Candidate Party']

    # Number of scenarios to consider
    n_rows = 6

    # Initialize data frame with these variables
    df_variables = pd.DataFrame(data=np.zeros((n_rows,len(rem_vars))), index=range(n_rows), columns=rem_vars)

    # Add column for scenario and district
    df_variables['District'] = [1, 1, 2, 2, 3, 3]
    df_variables['Scenario in District'] = [1, 2, 1, 2, 1, 2]
    df_variables['Scenario Probability'] = [0.7, 0.3, 0.4, 0.6, 0.8, 0.2]
    df_variables['Variable in District'] = ['Democrat Candidate Spending', 'Democrat Candidate Spending',
                                            'Republican Candidate Spending', 'Republican Candidate Spending',
                                            'Incumbent Candidate', 'Incumbent Candidate']

    # Initialize values in each column
    df_variables['Republican Candidate Spending'] = np.array([1000000, 2000000, 1000000, 3000000, 2000000, 3000000]) / 4000000
    df_variables['Democrat Candidate Spending'] = np.array([1500000, 4000000, 2000000, 4000000, 3000000, 2000000]) / 4000000
    df_variables['Party in Office'] = [0, 0, 1, 1, 0, 1]
    df_variables['Incumbent Republican Candidate Running'] = [1, 1, 0, 0, 1, 0]
    df_variables['Incumbent Democrat Candidate Running'] = [0, 0, 1, 1, 0, 1]
    df_variables['Candidate Party'] = np.ones(n_rows, dtype = int)

    # Generate all possible scenarios as binary lists
    scenario_list = list(itertools.product(*[(0, 1)] * 3))
    n_scenarios = len(scenario_list)

    # Initialize array to hold scenario values
    X_scenarios = np.empty((n_scenarios, num_districts, len(rem_vars) + X_district.shape[1]))

    # Initialize array to hold probability that each scenario occurs
    p_scenarios = np.empty(n_scenarios)

    # Fill array with values for all possible scenarios
    for i in range(n_scenarios):
        # Rows from data frame required for current scenario
        s_rows = [0,2,4] + np.asarray(scenario_list[i])
        # Probability of scenario i occuring
        p_scenarios[i] = np.prod(df_variables[['Scenario Probability']].iloc[s_rows].values)
        # Temp is array of variables from districts for current scenario
        temp = df_variables.drop(['District', 'Scenario in District', 'Variable in District', 'Scenario Probability'], axis=1).iloc[s_rows].values
        # Prepend district data with variable data for X_scenarios
        X_scenarios[i] = np.concatenate((temp, X_district), axis = 1)

    #print(p_scenarios)
    #print(X_scenarios)

    # Initialize names of binary files to save
    Xname = 'Numpy-Array-Files/Input-scenarios-Districts-' + str(district_number) + '.npy'
    pname = 'Numpy-Array-Files/Probability-scenarios-Districts-' + str(district_number) + '.npy'
    
    # Save both arrays to binary files
    np.save(Xname, X_scenarios)
    np.save(pname, p_scenarios)
    
    # Return number of scenarios and upper bounds for democrat spending
    return n_scenarios, X_scenarios[:,:,1].flatten()




# ------------------- Functions associated with minimizing Campign Expendetures ----------------------------- #
# ------------------------------- Objective function --------------------------------- #
def f (party_spending):
    # ----- Initialize variables to be used in evaluation ----- #
    # Set keras model filename location
    #keras_model_filename = 'Keras-Models/bin_model.h5'   # binary classification model
    #keras_model_filename = 'Keras-Models/dem_model_short_new.h5'
    keras_model_filename = 'Keras-Models/nnet_model.h5'
    # Set X_scens to contain the input data for all problem scenarios
    X_scens = np.load('Numpy-Array-Files/Input_scenarios.npy')
    # Set p_scens to contain the probability data for all problem scenarios
    P_scens = np.load('Numpy-Array-Files/Probability_scenarios.npy')
    # Set n_scens to be the number of scenarios
    n_scens = len(P_scens)
    # Set n_districts to be the number of districts
    n_districts = X_scens.shape[1]
    # Array to store values from evaluating keras_model
    district_outputs = np.array((n_scens, n_districts))
    
    # ----- Import new party spending values into scenario data ----- #
    # Update X_scens to reflect any changes in party spending (in this case changing amount of democrat spending)
    for i in range(n_scens):
        for j in range(n_districts):
            # Assign (n_districts*i + j)th element of party_spending to X_scens[i,j,1] ('1' is 'Democrat Spending' index)
            X_scens[i,j,1] = party_spending[n_districts * i + j]
    
    # ----- Set up keras model to evaluate function ----- #
    model = load_model(keras_model_filename)
            
    # ----- Compute objective value as expected value of probability democrat candidate will win ----- #
    # Initialize value to 0
    value = 0
    for i in range(n_scens):
        # Initialize temp to 0
        temp = 0
        # Evaluate keras model for current scenario data
        arr_temp = model.predict(X_scens[i])
        #print("arr_temp = ", arr_temp)

        # Sum all district values up in ith scenario
        for j in range(n_districts):
            # Add district_output value for jth district to temp
            #temp = temp + arr_temp[j][1]      # this line is using binary classification model
            temp = temp + arr_temp[j]        # this line if predicting percentage of votes

        # Multiply temp by probability scenario i occurs then add to value
        value = value + P_scens[i] * temp

    return -value
    
    
    
# ------------------- Gradient of Objective function ----------------------- #
def g (party_spending):
    # ----- Initialize variables to be used in evaluation ----- #
    # Set keras model filename location
    #keras_model_filename = 'Keras-Models/bin_model.h5'   # binary classification model
    #keras_model_filename = 'Keras-Models/dem_model_short_new.h5'
    keras_model_filename = 'Keras-Models/nnet_model.h5'
    # Set X_scens to contain the input data for all problem scenarios
    X_scens = np.load('Numpy-Array-Files/Input_scenarios.npy')
    # Set p_scens to contain the probability data for all problem scenarios
    P_scens = np.load('Numpy-Array-Files/Probability_scenarios.npy')
    # Set n_scens to be the number of scenarios
    n_scens = len(P_scens)
    # Set n_districts to be the number of districts
    n_districts = X_scens.shape[1]
    # Set n_features to be the number of features
    n_features = X_scens.shape[2]
    # Array to store values from evaluating keras_model
    district_outputs = np.array((n_scens, n_districts))
    
    # ----- Import new party spending values into scenario data ----- #
    # Update X_scens to reflect any changes in party spending (in this case changing amount of democrat spending)
    for i in range(n_scens):
        for j in range(n_districts):
            # Assign (n_districts*i + j)th element of party_spending to X_scens[i,j,1] ('1' is 'Democrat Spending' index)
            X_scens[i,j,1] = party_spending[n_districts * i + j]
    
    # ----- Set up keras model to evaluate gradient ----- #
    model = load_model(keras_model_filename)
    
    # Initialize output tensor for neural network model
    OutTensor = model.output

    # List of input variables for Tensor
    InVars = model.inputs
    #print(InVars)

    # Compute Gradients of model with respect to input variables
    Gradients = backend.gradients(OutTensor, InVars)
    #print(Gradients)

    # Evaluate Gradients
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
            
    # ----- Compute objective value as expected value of probability democrat candidate will win ----- #
    # Initialize grad to zero vector of length n_scenarios * n_districts
    grad = np.zeros(n_scens * n_districts)
    for i in range(n_scens):
        for j in range(n_districts):
            # Evaluate gradients at values for scenario i and district j
            eval_gradients = sess.run(Gradients, feed_dict={model.input: X_scens[i][j].reshape(1,n_features)})
            # Extract derivatives of objective with respect to Democratic and Republican Spending
            #repub_spending_derivative = eval_gradients[0][0][0]
            #dem_spending_derivative = eval_gradients[0][0][1]
            # Set (n_districts * i + j)th component of gradient value from tensorflow
            grad[n_districts*i + j] = - P_scens[i] * eval_gradients[0][0][1]
            
    # Close tensorflow session
    sess.close()
            
    return grad




# ------------------------------- Batch Objective function (more efficient computation) ------------------------------- #
def f_batch (party_spending, model, district_number):
    # ----- Initialize variables to be used in evaluation ----- #
    # Set keras model filename location
    #keras_model_filename = 'Keras-Models/bin_model.h5'   # binary classification model
    #keras_model_filename = 'Keras-Models/dem_model_short_new.h5'
    keras_model_filename = 'Keras-Models/nnet_model.h5'
    # Set filnames needed to load X and p data
    Xname = 'Numpy-Array-Files/Input-scenarios-Districts-' + str(district_number) + '.npy'
    pname = 'Numpy-Array-Files/Probability-scenarios-Districts-' + str(district_number) + '.npy'
    # Set X_scens to contain the input data for all problem scenarios
    X_scens = np.load(Xname)
    # Set p_scens to contain the probability data for all problem scenarios
    P_scens = np.load(pname)
    # Set n_scens to be the number of scenarios
    n_scens = len(P_scens)
    # Set n_districts to be the number of districts
    n_districts = X_scens.shape[1]
    # Array to store values from evaluating keras_model
    district_outputs = np.array((n_scens, n_districts))
    
    # ----- Import new party spending values into scenario data ----- #
    # Update X_scens to reflect any changes in party spending (in this case changing amount of democrat spending)
    for i in range(n_scens):
        for j in range(n_districts):
            # Assign (n_districts*i + j)th element of party_spending to X_scens[i,j,1] ('1' is 'Democrat Spending' index)
            X_scens[i,j,1] = party_spending[n_districts * i + j]
    
    # ----- Set up keras model to evaluate function ----- #
    #model = load_model(keras_model_filename)
            
    # ----- Compute objective value as expected value of probability democrat candidate will win ----- #
    # Initialize value to 0
    value = 0
    for i in range(n_scens):
        # Initialize temp to 0
        temp = 0
        # Evaluate keras model for current scenario data
        arr_temp = model.predict(X_scens[i])
        #print("arr_temp = ", arr_temp)

        # Sum all district values up in ith scenario
        for j in range(n_districts):
            # Add district_output value for jth district to temp
            #temp = temp + arr_temp[j][1]      # this line is using binary classification model
            temp = temp + arr_temp[j]        # this line if predicting percentage of votes

        # Multiply temp by probability scenario i occurs then add to value
        value = value + P_scens[i] * temp

    return -value



# ------------------- Batch Gradient of Objective function ----------------------- #
def g_batch (party_spending, Gradients, sess, batch):
    # ----- Initialize variables to be used in evaluation ----- #
    # Set keras model filename location
    #keras_model_filename = 'Keras-Models/bin_model.h5'   # binary classification model
    #keras_model_filename = 'Keras-Models/dem_model_short_new.h5'
    keras_model_filename = 'Keras-Models/nnet_model.h5'
    # Set filnames needed to load X and p data
    Xname = 'Numpy-Array-Files/Input-scenarios-Districts-' + str(district_number) + '.npy'
    pname = 'Numpy-Array-Files/Probability-scenarios-Districts-' + str(district_number) + '.npy'
    # Set X_scens to contain the input data for all problem scenarios
    X_scens = np.load(Xname)
    # Set p_scens to contain the probability data for all problem scenarios
    P_scens = np.load(pname)
    # Set n_scens to be the number of scenarios
    n_scens = len(P_scens)
    # Set n_districts to be the number of districts
    n_districts = X_scens.shape[1]
    # Set n_features to be the number of features
    n_features = X_scens.shape[2]
    # Array to store values from evaluating keras_model
    district_outputs = np.array((n_scens, n_districts))
    
    # ----- Import new party spending values into scenario data ----- #
    # Update X_scens to reflect any changes in party spending (in this case changing amount of democrat spending)
    for i in range(n_scens):
        for j in range(n_districts):
            # Assign (n_districts*i + j)th element of party_spending to X_scens[i,j,1] ('1' is 'Democrat Spending' index)
            X_scens[i,j,1] = party_spending[n_districts * i + j]
            
    # ----- Compute objective value as expected value of probability democrat candidate will win ----- #
    # Initialize grad to zero vector of length n_scenarios * n_districts
    grad = np.zeros(n_scens * n_districts)
    for i in batch:
        j = i % 3               # District number
        k = int((i - j)/n_districts) # Scenario number
        # Evaluate gradients at values for scenario i and district j
        eval_gradients = sess.run(Gradients, feed_dict={model.input: X_scens[k][j].reshape(1,n_features)})
        # Extract derivatives of objective with respect to Democratic and Republican Spending
        #repub_spending_derivative = eval_gradients[0][0][0]
        #dem_spending_derivative = eval_gradients[0][0][1]
        # Set (n_districts * i + j)th component of gradient value from tensorflow
        grad[i] = - P_scens[k] * eval_gradients[0][0][1]
            
    return grad


# ----------- Stochastic Gradient Descent for Objective function ---------- #
def stoch_grad_proj(f_batch, g_batch, Gradients, sess, x, bounds, batch_size, n_scens, line_search, model, dist_number):
    # Generate indices to keep for gradient descent
    batch = np.random.choice(n_scens, batch_size, replace = False)
    #print("Current batch: ", batch)
    
    # ------------ Compute gradient values --------- #    
    # Initialize gradient values
    #start = time.time()
    gradient = np.copy(g_batch(x, Gradients, sess, batch))
    #end = time.time()
    #print("Gradient Time elapsed: ", end - start)
    # Set components of gradient not in current batch equal to zero
    for i in range(n_scens):
        if i not in batch:
            gradient[i] = 0
    
    # ------------ Line Search (if requested) --------- #
    # Initialize alpha parameter for line search
    alpha = 1
    # Check if line search was requested
    if line_search == True: 
        # Initialize parameter for line search
        c = 0.5
        # Perform armijo line search
        while f_batch(x, model, dist_number) - f_batch(x - alpha * gradient, model, dist_number) < - alpha * c * np.dot(gradient, gradient):
            # Reduce alpha size and try again
            alpha = 0.5 * alpha
    
    # Update x value
    xnew = x - alpha * gradient
    
    # ------------ Gradient Projection --------- #
    # Project values of x back onto bounds if necessary
    for i in batch:
        # Check if xnew is less than lower bound
        if xnew[i] < bounds[i][0]:
            #print("below lower")
            # Set xnew to lower bound
            xnew[i] = bounds[i][0]
        # Check if xnew exceeds upper bound
        elif xnew[i] > bounds[i][1]:
            #print("exceeds upper")
            # Set xnew to upper bound
            xnew[i] = bounds[i][1]
            
    # ---------- Return new x value ------------ #
    return xnew

In [ ]:
# For every county in our state, generate uniformly distributed points in county representing 0.1 percent of population in county
# Set number of districts
num_districts = 3
# Set number of counties
num_counties = 37
# Set number of races
num_races = 2

# Generate uniformly distributed population data
county_pop = df_florida[['Total; Estimate; Total population']].values
county_pop = county_pop.flatten()
#county_pop = np.random.permutation(county_pop)
#print(county_pop)
grid_scale = 100000 # x and y dimensions of each county
pop_scale = 1000  # number of people each point on the plot corresponds to


# Determine total population
total_pop = int(sum(county_pop))

# Initialize arrays for x, y coordinates and race percentages of counties
x = []
y = []
county_array = []
R = [[] for i in range(num_races)]

# --------------- Extract race percentages for each county ------------------ #
race_list = ['Total; Estimate; RACE AND HISPANIC OR LATINO ORIGIN - One race - White',
             'Total; Estimate; RACE AND HISPANIC OR LATINO ORIGIN - One race - Black or African American']

# Initialize data frames containing race data
df_races = df_florida[race_list].copy()

# Convert each value from raw value to decimal (representing percentage)
for mystr in race_list:
    # Convert percentage to value
    df_races[mystr] = df_races[mystr] / df_florida['Total; Estimate; Total population']

# Create array containing race distributions
R_dist = df_races.values.T
# To use randomized race data, use the following command to generate R_dist
#R_dist = np.random.dirichlet(5*np.ones(num_races), num_counties).T

# Initialize dimensions for state
dimen = math.ceil(math.sqrt(num_counties))

# Populate the arrays x, y, and R using randomly generated data
county_num = 0

for i in range(dimen):
    # Initialize population distribution data
    for j in range(dimen):
        num_points = int(county_pop[county_num]/pop_scale)
        t1 = np.random.uniform(grid_scale*i,grid_scale*(i+1),num_points)
        t2 = np.random.uniform(grid_scale*j,grid_scale*(j+1),num_points)
        x = np.append(x, t1)
        y = np.append(y, t2)
        county_array = np.append(county_array, county_num*np.ones(num_points))
        # Initialize race distribution data
        for k in range(num_races):
            temp = np.ones(num_points)*R_dist[k][j]
            R[k] = np.append(R[k], temp)
        # Update counties done counter
        county_num = county_num + 1
        # Check if all counties have been populated
        if county_num >= num_counties:
            break;
    # Check if all counties have been populated
    if county_num >= num_counties:
        break;
            
# Convert each numpy array in R to a list
for k in range(num_races):
    R[k] = R[k].tolist()
            
# Generate mean of race percentages in state
mu_R = []
for i in range(num_races):
    mu_R = np.append(mu_R, np.mean(R[i])) 
    
# Combine x and y into vector of ordered pairs
P = np.column_stack((x,y))

# Determine population per district
points_per_dist = int(len(P)/num_districts)

In [ ]:
# Use the following point to generate random starting points for districts
#pt = np.random.uniform(0,600000,(num_districts,2))
#print(pt)

# Set of points for generating districts
pts = np.array([[[428596.73182419, 565472.09203069], [406481.47389631,  76214.63431997], [529037.777425,   301312.41034136]],
                [[103815.07274117, 523223.20943294], [127784.24278143, 367946.5601808 ], [405265.50666261, 126535.48730887]],
                [[232498.50976297, 333876.44175454], [434856.49421111, 536007.79553189], [304201.82051287,  51653.77351857]],
                [[264530.56995353, 453923.23875946], [439665.38813067, 440157.66435593], [129802.05652538, 421065.93395801]],
                [[577596.93197487, 221554.65871974], [393323.71962208, 576595.73225033], [356491.77699861, 193253.70606583]],
                [[ 10069.19911696,  27137.21980555], [541840.3264807,    4792.29529087], [134325.7114619,  582525.18892094]],
                [[345564.19987199, 529750.04305769], [354862.82145347, 245082.66827711], [473278.58737241,   7581.06023634]],
                [[123099.25101014,   6655.89064585], [ 18575.57679205, 245189.24132337], [321515.0611377,  530000.62871451]]])

#print(pts)

# Number or nearest neighbors to use in initial construction of districts
number_of_neighbors = 4

# Set point name
district_number = 0
i = district_number

# Loop over all initial district points and generate local minimums
for i in range(8):
    # Generate district and return x, y coordinates, indices of points in P and mean of races in each district
    xdk, ydk, indk, r_mean = construct_district(num_districts, number_of_neighbors, pts[i], P, R, 1)

    # Generate nearest neighbors model
    nn_model = initial_district_classification(P, indk, i)

    # Generate district from nearest neighbor classification
    Y_dist = final_district_classification(P, nn_model, i)

    # Using district information, compile County data into District data
    n_scenarios, budget_max = county_to_district(df_florida, df_florida_header_list, num_districts, num_counties, county_array, Y_dist, i)

    # ----- Compute local minimum for district ----- #
    # Set number of decision variables
    n_dec_vars = n_scenarios * num_districts

    # Define bound constraints on the variables
    budget_bounds = np.stack((np.zeros(n_dec_vars), budget_max), axis = -1)

    # ----- Set up keras model to evaluate gradient ----- #
    # Initialize tensorflow session first
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    # Initialize keras model from training
    keras_model_filename = 'Keras-Models/nnet_model.h5'
    model = load_model(keras_model_filename)

    # Initialize output tensor for neural network model
    OutTensor = model.output

    # List of input variables for Tensor
    InVars = model.inputs
    #print(InVars)

    # Compute Gradients of model with respect to input variables
    Gradients = backend.gradients(OutTensor, InVars)
    #print(Gradients)

    # Set filename for writing data
    file_name = 'Finalized-Districts/Local-Minimizer/Districts-' + str(i) + '-local-minimizer.txt'
    file = open(file_name, 'w')

    # Find three different local minimums
    for j in range(3):
        # Set initial guess
        print("District %i - Point %i" %(i,j))
        #x0 = np.random.uniform(0, 1, X_scenarios.shape[0] * X_scenarios.shape[1])
        x0 = (3-j) * np.ones(n_dec_vars) / 3
        # Run stochastic gradient projection on objective function
        #start = time.time()
        for k in range(2001):
            # Most of the time run the stochastic gradient projection with line search
            if k % 100 != 0:
                x0 = stoch_grad_proj(f_batch, g_batch, Gradients, sess, x0, budget_bounds, 5, n_dec_vars, True, model, i)
            else:
                x0 = stoch_grad_proj(f_batch, g_batch, Gradients, sess, x0, budget_bounds, 5, n_dec_vars, False, model, i)
            # Print out values every 400th iteration
            if k % 400 == 0:
                # End timer
                #end = time.time()
                # Print current statistics
                #print("Time elapsed: ", end - start)
                #print("x_", k, ":", x0)
                print("f(x_", k, ") =", f_batch(x0, model, i))
                # Reset timer
                #start = time.time()

        # Make filename to save solution point
        local_min_name = 'Numpy-Array-Files/' + str(i) + 'local_minimizer' + str(j) + '.npy'

        # Save final iterate
        np.save(local_min_name, x0)

        # Convert values in final iterate to dollar amounts by multiplying by 4000000
        x_dollars = 4000000 * x0

        # Print statistics for state population
        file.write("-------- District %i - Solution %i --------\n" %(i,j))
        file.write("x    = ")
        for l in range(len(x_dollars)):
            file.write(str(round(x_dollars[l], 2)))
            file.write(", ")
        file.write("\n")
        file.write("f(x) = ")
        file.write(str(f_batch(x0, model, i)[0]))
        file.write("\n\n")

    # Close file
    file.close()

    # Close tensorflow session
    sess.close()


# Print means of races in districts and actual mean of races from population data
#print(r_mean)
#print(mu_R) 

In [ ]:
#mm_model = initial_district_classification(P, indk, i)

# Points that have worked well so far in generating contiguous districts
#pt = [[428596.73182419 565472.09203069], [406481.47389631  76214.63431997], [529037.777425   301312.41034136]]
#pt = [[103815.07274117 523223.20943294], [127784.24278143 367946.5601808 ], [405265.50666261 126535.48730887]]
#pt = [[232498.50976297 333876.44175454], [434856.49421111 536007.79553189], [304201.82051287  51653.77351857]]
#pt = [[264530.56995353 453923.23875946], [439665.38813067 440157.66435593], [129802.05652538 421065.93395801]]
#pt = [[577596.93197487 221554.65871974], [393323.71962208 576595.73225033], [356491.77699861 193253.70606583]]
#pt = [[ 10069.19911696  27137.21980555], [541840.3264807    4792.29529087], [134325.7114619  582525.18892094]]
#pt = [[345564.19987199 529750.04305769], [354862.82145347 245082.66827711], [473278.58737241   7581.06023634]]
#pt = [[123099.25101014   6655.89064585], [ 18575.57679205 245189.24132337], [321515.0611377  530000.62871451]]


# ------ Found when using randomized race data (not sure if good for actual data)
#pt = [[380002.71426247 342680.65629285], [367484.54563801 156618.81390881], [420636.80961064 508608.51930695]]
#pt = [[350738.54781446 432803.9468541 ], [ 14892.65777381 133318.63679456], [225162.24348173 188512.12569184]]
#pt = [[ 42186.45717472 583606.98205745], [492200.10479782 312295.99733307], [295188.97835625 272046.10439235]]

# Extra code leftover from testing
# ----------------- Generate sample plot of initial data points ----------------- #
fig = plt.figure(figsize=(20, 20))
#ax2 = fig.add_subplot(111, aspect='equal')
#for p in [
#    patches.Rectangle((0, 0), 1000, 1000, fill=False),
#    patches.Rectangle((0, 1000), 1000, 2000, fill=False),
#]:
#    ax2.add_patch(p)
plt.plot(x,y,'bo')
plt.show()

# ----------------- Testing objective function and gradients -------------------- #
party_spend = np.random.uniform(0, 1, X_scenarios.shape[0] * X_scenarios.shape[1])
print(party_spend)

sdf = f(party_spend)

print("objective value = ", sdf)

sdf = g(party_spend)

print(sdf)